# **SETUP**

In [ ]:
import sys
from pathlib import Path
sys.path.append('/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/src') 
from config import *

# Fixed PATHs
# NET_XML = Path("/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/data/newtest-osm.net.xml")
NET_XML = "/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/result/net-creation/310525-AMP-test-func/connected-network.net.xml"
# VTYPES_DIST_XML = "/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/data/additional-files/vtypes-dist.add.xml"
TAZ_XML = "/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/result/net-creation/310525-AMP-test-func/taz.xml"
VTYPES_XML = "/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/data/additional-files/vtypes/eidm-1.xml"
ODS_DIR = "/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/data/processed-data/total_scenario/old-form-all"

# Dynamic DIRs
SIMULATION_DIR = Path("/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/result/experiments/02-06-new-scenario") # MOST IMPORTANT
TRIPS_DIR = SIMULATION_DIR / "trips"
OUTPUTS_DIR = SIMULATION_DIR / "outputs"
LOGS_DIR = SIMULATION_DIR / "logs"
VISUALIZATIONS_DIR = SIMULATION_DIR / "visualizations"

SIMULATION_DIR.mkdir(parents=True, exist_ok=True)
for path in [TRIPS_DIR, OUTPUTS_DIR, LOGS_DIR, VISUALIZATIONS_DIR]:
    path.mkdir(parents=True, exist_ok=True)

# Dynamic PATHs
ALL_TRIPS_XML = SIMULATION_DIR / "trips.xml"
ROUTE_XML = SIMULATION_DIR / "route.xml"
ROUTE_ALT_XML = SIMULATION_DIR / "route.alt.xml"
SUMOCFG_XML = SIMULATION_DIR / "run.sumocfg"

DUAROUTER_LOG = LOGS_DIR / "duarouter.log"
SIMULATION_LOG = LOGS_DIR / "sumo_run.log"

# Outputs Paths
COLLISIONS_XML = OUTPUTS_DIR / "collisions.xml"
BATTERY_XML = OUTPUTS_DIR / "battery.xml"
LANECHANGES_XML = OUTPUTS_DIR / "laneChanges.xml"
STATISTICS_XML = OUTPUTS_DIR / "statistics.xml"
TRACE_XML = OUTPUTS_DIR / "sumoTrace.xml"
SUMMARY_XML = OUTPUTS_DIR / "summary.xml"
TRIPINFO_XML = OUTPUTS_DIR / "tripinfo.xml"
VEHROUTES_XML = OUTPUTS_DIR / "vehRoutes.xml"
NETSTATE_XML = OUTPUTS_DIR / "netstate.xml"
LOG_TXT = OUTPUTS_DIR / "log.txt"
EDGEDATA_XML = OUTPUTS_DIR / "edgeData-60s.xml"
LANEDATA_XML = OUTPUTS_DIR / "laneData-60s.xml"
QUEUE_XML = OUTPUTS_DIR / "queue.xml"
EMISSION_XML = OUTPUTS_DIR / "emission.xml"
SMS_XML = OUTPUTS_DIR / "sms.xml"

# Visualization Paths
PLOT_1_PNG = VISUALIZATIONS_DIR / "plot_1.png"
PLOT_2_PNG = VISUALIZATIONS_DIR / "plot_2.png"
PLOT_3_PNG = VISUALIZATIONS_DIR / "plot_3.png"
PLOT_4_PNG = VISUALIZATIONS_DIR / "plot_4.png"
PLOT_5_PNG = VISUALIZATIONS_DIR / "plot_5.png"
PLOT_6_PNG = VISUALIZATIONS_DIR / "plot_6.png"
PLOT_7_PNG = VISUALIZATIONS_DIR / "plot_7.png"
PLOT_8_PNG = VISUALIZATIONS_DIR / "plot_8.png"

PATH_REPLACEMENTS = {
    'net-file': NET_XML,
    'route-files': ROUTE_XML,
    'summary-output': SUMMARY_XML,
    'tripinfo-output': TRIPINFO_XML,
    'fcd-output': TRACE_XML,
    'lanechange-output': LANECHANGES_XML,
    'battery-output': BATTERY_XML,
    'vehroute-output': VEHROUTES_XML,
    'collision-output': COLLISIONS_XML,
    'netstate-dump': NETSTATE_XML,
    'statistic-output': STATISTICS_XML,
    'log': LOG_TXT,
    'lanedata-output': LANEDATA_XML,
    'edgedata-output': EDGEDATA_XML,
    'device.ssm.file': SMS_XML,
    'emission-output': EMISSION_XML,
    'queue-output': QUEUE_XML
}

# **MAIN**

## Creating Trips from Ods

In [ ]:
print("Call od2trips for all ...")
trips_files = od2tripsForAll(TAZ_XML, TRIPS_DIR, ODS_DIR, "car_eidm")
print()
print("[DONE] Finished 24 Trips based on hours.")

mergeTrips(TRIPS_DIR, ALL_TRIPS_XML)


Call od2trips for all ...
Success.time 3598.38
Generated /home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/result/experiments/02-06-new-scenario/trips/trips_00.xml
Success.time 7197.29
Generated /home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/result/experiments/02-06-new-scenario/trips/trips_01.xml
Success.time 10792.44
Generated /home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/result/experiments/02-06-new-scenario/trips/trips_02.xml
Success.time 14392.68
Generated /home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/result/experiments/02-06-new-scenario/trips/trips_03.xml
Success.time 17997.88
Generated /home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/result/experiments/02-06-new-scenario/trips/trips_04.xml
Success.time 21599.60
Generated /home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/result/experiments/02-06-new-scenario/trips/trips_05.xml
Success.time 25199.68.22
Generated /home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/result/experiments/02-06-new-scenario/trips/trips_06.xml
Success.time 28799.2454.29g time 26520.69 27841.5

## Creating Route

In [1]:
DUAROUTER_CMD = [
    "duarouter",
    "-n", NET_XML,            
    "-r", ALL_TRIPS_XML,            
    "-a", VTYPES_XML,
    # "--randomize-flows",         
    "-o", ROUTE_XML,    
    "--log", DUAROUTER_LOG,
    "--exit-times",
    "--named-routes",
    "--route-length",
    "--write-costs",
    "--ignore-errors",
    "--seed", "2005"
]

print("Running DUAROUTER Step ...")
subprocess.run(DUAROUTER_CMD, check=True)
print(f"[DONE] Routes written in {ROUTE_XML}\n[LOG] Output logged in {DUAROUTER_LOG}")


NameError: name 'NET_XML' is not defined

## Updating Sumo-Config

In [ ]:
updateSumoCfg(
    cfg_path=SUMOCFG_XML,
    output_path=SUMOCFG_XML,
    replacements=PATH_REPLACEMENTS
)
print(f"[DONE] New Sumo-Configuration in {SUMOCFG_XML} with all right paths.")


Updating net-file: /home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/result/net-repairment/cleaned_p2_2_newtest-osm.net.xml -> /home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/result/net-repairment/cleaned_p2_2_newtest-osm.net.xml
Updating route-files: /home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/result/experiments/19-05-25-debugging/route.xml -> /home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/result/experiments/19-05-25-debugging/route.xml
Updating statistic-output: /home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/result/experiments/19-05-25-debugging/outputs/statistics.xml -> /home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/result/experiments/19-05-25-debugging/outputs/statistics.xml
Updating log: /home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/result/experiments/19-05-25-debugging/outputs/log.txt -> /home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/result/experiments/19-05-25-debugging/outputs/log.txt
Updating summary-output: /home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/result/experiments/19-05-25-debugging/outputs/summary.xml 

## Running

In [ ]:
SIMULATION_CMD = [
    "sumo",             
    "-c", SUMOCFG_XML,
    "--no-step-log",      
    "--duration-log.statistics",
    "--xml-validation", "never"  
]

with open(SIMULATION_LOG, "w") as f:
    print("Running SUMO simulation ...")
    subprocess.run(SIMULATION_CMD, stdout=f, stderr=subprocess.STDOUT, check=True)
    print(f"[DONE] Simulation outputs are created in {SIMULATION_DIR}\n[LOG] Output logged in {SIMULATION_LOG}")


Running SUMO simulation ...


# **VISUALIZATIONS**

In [ ]:
# Departure times versus arrival times
PLOT_CMD_1 = [
    "python", PLOTXMLATTRIBUTES_PY,
    VEHROUTES_XML,                 
    "-x", "depart",      
    "-y", "arrival",             
    "-o", PLOT_1_PNG,
    "--scatterplot"
]

subprocess.run(PLOT_CMD_1, check=True)


In [ ]:
# All trajectories over time 1
PLOT_CMD_2 = [
    "python", PLOTXMLATTRIBUTES_PY,
    TRACE_XML,                 
    "-x", "x",     
    "-y", "y",             
    "-o", PLOT_2_PNG,
    "--scatterplot"
]

subprocess.run(PLOT_CMD_2, check=True)


In [ ]:
# Multiple timelines from summary-output
PLOT_CMD_3 = [
    "python", PLOTXMLATTRIBUTES_PY,
    SUMMARY_XML,
    "-x", "time",
    "-y", "running,halting",
    "-o", PLOT_3_PNG,
    "--legend"
]
subprocess.run(PLOT_CMD_3, check=True)


In [ ]:
# Depart delay over time from TripInfo data
PLOT_CMD_4 = [
    "python", PLOTXMLATTRIBUTES_PY,
    TRIPINFO_XML,
    "-x", "depart",
    "-y", "departDelay",
    "--xlabel", "depart time [s]",
    "--ylabel", "depart delay [s]",
    "--ylim", "0,40",
    "--xticks", "0,1200,200,10",
    "--yticks", "0,40,5,10",
    "--xgrid", "--ygrid",
    "--title", "depart delay over depart time",
    "--titlesize", "16",
    "-o", PLOT_4_PNG
]
subprocess.run(PLOT_CMD_4, check=True)


In [ ]:
QUERIED_VEH_ID = "carDist1"
# Selected trajectories over time
PLOT_CMD_5 = [
    "python", PLOTXMLATTRIBUTES_PY,
    TRACE_XML,
    "-x", "x",
    "-y", "y",
    "-i", "id",
    "--filter-ids", QUERIED_VEH_ID
    "--scatterplot",
    "--legend",
    "-o", PLOT_5_PNG
]
subprocess.run(PLOT_CMD_4, check=True)